In [1]:
!pwd

/home/giakhang/dev/pose_sandbox


In [2]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.curdir),
                "Hand_pose_estimation_3D/arm_and_hand"))
sys.path.append(os.path.join(os.path.abspath(os.curdir),
                "Hand_pose_estimation_3D"))

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
from datetime import datetime
import glob
from torch.optim.lr_scheduler import ReduceLROnPlateau
from ann import ANN
import math
from sklearn.preprocessing import MinMaxScaler
from csv_writer import columns_to_normalize, fusion_csv_columns_name
import pandas as pd
import joblib
import numpy as np

from landmarks_scaler import LandmarksScaler
from train_ann_with_anchors import train_model
from dataloader_hand_only_ann_with_anchors import HandLandmarksDataset_ANN_With_Anchors

In [3]:
num_hand_lmks = 21
num_hand_anchors = 3

In [4]:
#INPUT_DIM = (num_hand_lmks * 3 * 2) + (5 * 3)  + (num_hand_anchors * 3)
INPUT_DIM = (num_hand_lmks * 3 * 2) 
OUTPUT_DIM = (num_hand_lmks - num_hand_anchors) * 3
HIDDEN_DIM_CONTAINER = [num_hand_lmks * 3 * 2, 100, 100, 100, 70, 70]
NUM_HIDDEN_LAYERS = 5
DROPOUT_RATE = 0.1

model = ANN(input_dim=INPUT_DIM,
            output_dim=OUTPUT_DIM,
            hidden_dim_container=HIDDEN_DIM_CONTAINER,
            num_hidden_layers=NUM_HIDDEN_LAYERS,
            dropout_rate=DROPOUT_RATE)
model = model.to("cuda")

In [5]:
MODEL_NAME = "ann_left_hand"
DATETIME = "{}".format(datetime.now().strftime("%Y%m%d-%H%M"))
DATE = "{}".format(datetime.now().strftime("%Y%m%d"))
BASE_DIR = "Hand_pose_estimation_3D/arm_and_hand/runs/{}".format(MODEL_NAME)
SAVE_DIR = os.path.join(BASE_DIR, DATE, DATETIME)
DATA_DIR = "data"  
writer = SummaryWriter(log_dir=SAVE_DIR)

In [6]:
arm_hand_fused_names = ["left shoulder", "left elbow", "left hip", "right shoulder",
    "right hip", "WRIST", "THUMB_CMC", "THUMB_MCP", "THUMB_IP", 
    "THUMB_TIP", "INDEX_FINGER_MCP", "INDEX_FINGER_PIP", "INDEX_FINGER_DIP",
    "INDEX_FINGER_TIP", "MIDDLE_FINGER_MCP", "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP",
    "MIDDLE_FINGER_TIP", "RING_FINGER_MCP", "RING_FINGER_PIP", "RING_FINGER_DIP",
    "RING_FINGER_TIP", "PINKY_MCP", "PINKY_PIP", "PINKY_DIP", "PINKY_TIP", "right elbow",
    "RIGHT_WRIST", "RIGHT_THUMB_CMC", "RIGHT_THUMB_MCP", "RIGHT_THUMB_IP", "RIGHT_THUMB_TIP",
    "RIGHT_INDEX_FINGER_MCP", "RIGHT_INDEX_FINGER_PIP", "RIGHT_INDEX_FINGER_DIP",
    "RIGHT_INDEX_FINGER_TIP", "RIGHT_MIDDLE_FINGER_MCP", "RIGHT_MIDDLE_FINGER_PIP",
    "RIGHT_MIDDLE_FINGER_DIP", "RIGHT_MIDDLE_FINGER_TIP", "RIGHT_RING_FINGER_MCP",
    "RIGHT_RING_FINGER_PIP", "RIGHT_RING_FINGER_DIP", "RIGHT_RING_FINGER_TIP",
    "RIGHT_PINKY_MCP", "RIGHT_PINKY_PIP", "RIGHT_PINKY_DIP", "RIGHT_PINKY_TIP"]

In [7]:
DATA_DIR = "data"  
SELECTED_DATE = "2024-*"  # Keep '*' when using glob.glob

train_paths = glob.glob(os.path.join(DATA_DIR, "{}/{}/fine_landmarks_{}_*.csv".format(SELECTED_DATE, SELECTED_DATE, "train")))
val_paths = glob.glob(os.path.join(DATA_DIR, "{}/{}/fine_landmarks_{}_*.csv".format(SELECTED_DATE, SELECTED_DATE, "val")))

body_lines = [[0,2], [0, 3], [2, 4], [3, 4]]
lefthand_lines = [[0, 1], [1, 5], [5, 6], [5, 10], [5, 22], [10, 14], [14, 18], [18, 22], 
    [6, 7], [7, 8], [8, 9], 
    [10, 11], [11, 12], [12, 13], 
    [14, 15], [15, 16], [16, 17], 
    [18, 19], [19, 20], [20, 21], 
    [22, 23], [23, 24], [24, 25]]
train_body_distance_thres = 550
train_leftarm_distance_thres = 550
train_lefthand_distance_thres = 200
val_body_distance_thres = 450
val_leftarm_distance_thres = 450
val_lefthand_distance_thres = 150

input_scaler = MinMaxScaler()
output_scaler = MinMaxScaler()

train_dataset = HandLandmarksDataset_ANN_With_Anchors(train_paths, 
    arm_hand_fused_names,
    body_lines, 
    lefthand_lines, 
    train_body_distance_thres, 
    train_leftarm_distance_thres, 
    train_lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True,
    cvt_normalized_xy_to_XY=True,
    use_thumb_as_anchor=False)

In [8]:
input_scaler.fit_transform(train_dataset._inputs)
output_scaler.fit_transform(train_dataset._outputs)

input_scaler_save_path = os.path.join(SAVE_DIR, "input_scaler.pkl")
output_scaler_save_path = os.path.join(SAVE_DIR, "output_scaler.pkl")

joblib.dump(input_scaler, input_scaler_save_path)
joblib.dump(output_scaler, output_scaler_save_path)

['Hand_pose_estimation_3D/arm_and_hand/runs/ann_left_hand/20241108/20241108-1450/output_scaler.pkl']

In [9]:
input_scaler = LandmarksScaler(scaler_path=input_scaler_save_path)
output_scaler = LandmarksScaler(scaler_path=output_scaler_save_path)

train_dataset = HandLandmarksDataset_ANN_With_Anchors(train_paths, 
    arm_hand_fused_names,
    body_lines, 
    lefthand_lines, 
    train_body_distance_thres, 
    train_leftarm_distance_thres, 
    train_lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True,
    cvt_normalized_xy_to_XY=True,
    use_thumb_as_anchor=False,
    input_scaler=input_scaler,
    output_scaler=output_scaler)
train_dataloader = DataLoader(train_dataset, batch_size=20000, shuffle=True)

In [10]:
val_dataset = HandLandmarksDataset_ANN_With_Anchors(val_paths,
    arm_hand_fused_names,
    body_lines,
    lefthand_lines,
    val_body_distance_thres,
    val_leftarm_distance_thres,
    val_lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True,
    cvt_normalized_xy_to_XY=True,
    use_thumb_as_anchor=False,
    input_scaler=input_scaler,
    output_scaler=output_scaler)
val_dataloader = DataLoader(val_dataset, batch_size=1000, shuffle=True)  

In [11]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 10000
current_time = datetime.now().strftime('%Y%m%d-%H%M')
save_path = os.path.join(SAVE_DIR, "{}_{}_layers_best.pth".format(MODEL_NAME, NUM_HIDDEN_LAYERS))
scheduler = ReduceLROnPlateau(optimizer, mode='min', 
    factor=math.sqrt(0.1), patience=500, verbose=True, min_lr=1e-9)

In [ ]:
train_losses, val_losses = train_model(model, 
    train_dataloader, 
    val_dataloader, 
    optimizer, 
    num_epochs=num_epochs, 
    save_path=save_path,
    scheduler=scheduler,
    writer=writer,
    log_seq=50)

writer.close()

Model saved with Validation Loss: 0.9352
Model saved with Validation Loss: 0.7429
Model saved with Validation Loss: 0.6085
Model saved with Validation Loss: 0.5157
Model saved with Validation Loss: 0.4573
Model saved with Validation Loss: 0.4216
Model saved with Validation Loss: 0.3973
Model saved with Validation Loss: 0.3815
Model saved with Validation Loss: 0.3704
Model saved with Validation Loss: 0.3621
Model saved with Validation Loss: 0.3538
Model saved with Validation Loss: 0.3479
Model saved with Validation Loss: 0.3429
Model saved with Validation Loss: 0.3395
Model saved with Validation Loss: 0.3349
Model saved with Validation Loss: 0.3312
Model saved with Validation Loss: 0.3306
Model saved with Validation Loss: 0.3253
Model saved with Validation Loss: 0.3242
Model saved with Validation Loss: 0.3223
Model saved with Validation Loss: 0.3171
Model saved with Validation Loss: 0.3096
Model saved with Validation Loss: 0.3056
Model saved with Validation Loss: 0.2999
Model saved with

KeyboardInterrupt: 